# Notebook for making comparison between my solvers and Yandex solver

In [46]:
from lib.solver_tools.solver_client import SolverClient
from lib.solver_tools import Task
from lib.solver_tools.utils import make_task_comparison_link
import json
from copy import deepcopy
from lib.common_tools.read_token import read_solver_apikey
import os

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(os.path.join(os.getcwd(), '..'))
import python_lib.viz as my_viz
import python_lib.utils as my_utils

In [47]:
my_apikey = read_solver_apikey()
solver_client = SolverClient(my_apikey)

In [49]:
request_path = '../test_data/inputs/simple_test_1/request.json'
request = my_utils.read_json(request_path)

In [50]:
task_id = solver_client.add_task_and_get_task_id(request)

In [60]:
Task(task_id).map()

Neither solver_api_key nor ya_courier_token provided to SolverClient. Using client in limited functionality mode


In [53]:
path_to_result_directory = '../test_data/results/annealing/simple_test_1'
path_to_checkpoints = os.path.join(path_to_result_directory, 'checkpoints_1.json')
checkpoints = read_json(path_to_checkpoints)

In [54]:
def get_request_with_fixed_routes(request, routes):
    # routes must contain fields 'vehicle_id', 'location_ids'
    new_request = deepcopy(request)
    vehicle_id_to_location_ids = {route['vehicle_id'] : route['location_ids'] for route in routes}
    for vehicle in new_request['vehicles']:
        vehicle['visited_locations'] = [{'id': location_id} for location_id in vehicle_id_to_location_ids[vehicle['id']]]
    return new_request

In [55]:
request_with_fixed_routes = get_request_with_fixed_routes(request, checkpoints[-1]['routes'])

In [56]:
my_task_id = solver_client.add_task_and_get_task_id(request_with_fixed_routes)

/b2bgeo/environments/common_env/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'courier.yandex.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [59]:
Task(my_task_id).map()

Neither solver_api_key nor ya_courier_token provided to SolverClient. Using client in limited functionality mode


In [58]:
make_task_comparison_link([task_id, my_task_id])